# We use mnist as training dataset and fashion-mnist as anomalous data.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Conv2DTranspose, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import mnist, fashion_mnist

## Loading datasets

In [13]:
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()

X_train_mnist = X_train_mnist / 255.0
X_test_mnist = X_test_mnist / 255.0

In [14]:
(X_train_fashion, y_train_fashion), (X_test_fashion, y_test_fashion) = fashion_mnist.load_data()

X_train_fashion = X_train_fashion / 255.0
X_test_fashion = X_test_fashion / 255.0

## Defining autoencoder

In [15]:
encoder = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization()
])

In [16]:
decoder = Sequential([
    Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', activation='sigmoid'), # We use sigmoid here as it normalizes output -> (0, 1)
])

### Loss function used for comparing images is SSIM (Structured Similarity Index). It compares images based on structure, luminance and contrast.

In [17]:
# Defining the SSIM loss function
def SSIMLoss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

## Training the model

In [18]:
autoencoder = Sequential([
    encoder,
    decoder
])

In [19]:
autoencoder.compile(optimizer='adam', loss=SSIMLoss)

In [20]:
autoencoder.fit(X_train_mnist, X_train_mnist, epochs=10, shuffle=True, validation_data=(X_test_mnist, X_test_mnist))

Epoch 1/10
1875/1875 [==============================] - 41s 21ms/step - loss: 0.1351 - val_loss: 0.1148
Epoch 2/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1116 - val_loss: 0.1099
Epoch 3/10
1875/1875 [==============================] - 42s 22ms/step - loss: 0.1078 - val_loss: 0.0777
Epoch 4/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0736 - val_loss: 0.0732
Epoch 5/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0731 - val_loss: 0.0753
Epoch 6/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.0727 - val_loss: 0.0726
Epoch 7/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0725 - val_loss: 0.0723
Epoch 8/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0724 - val_loss: 0.0724
Epoch 9/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0722 - val_loss: 0.0724
Epoch 10/10
1875/1875 [==============================] - 37s 20m

In [30]:
autoencoder.evaluate(X_test_mnist, X_test_mnist)

313/313 [==============================] - 2s 6ms/step - loss: 0.0720


0.07203537970781326

## Testing on fashion-mnist dataset

In [34]:
reconstructed_images = autoencoder.predict(X_train_fashion.reshape(-1, 28, 28, 1))

1875/1875 [==============================] - 11s 6ms/step


In [35]:
print(f'SSIM Loss: {SSIMLoss(X_train_fashion.reshape(-1, 28, 28, 1), reconstructed_image)}')

SSIM Loss: 0.9656606316566467
